In [1]:
import csv

In [47]:
# create a class that can store the data from the file
# focus on the attributes only
class Shipments:
    def __init__(self, date_received, ingredient_name, supplier, number_of_cases, units_per_case,
                 sample_case_count1, sample_case_count2, sample_case_count3,
                 sample_case_rejections1, sample_case_rejections2, sample_case_rejections3):
        self.date_received = date_received
        self.ingredient_name = ingredient_name
        self.supplier = supplier
        self.number_of_cases = int(number_of_cases)
        self.units_per_case = int(units_per_case)
        self.sample_case_count1 = int(sample_case_count1)
        self.sample_case_count2 = int(sample_case_count2)
        self.sample_case_count3 = int(sample_case_count3)
        self.sample_case_rejections1 = int(sample_case_rejections1)
        self.sample_case_rejections2 = int(sample_case_rejections2)
        self.sample_case_rejections3 = int(sample_case_rejections3)
        self.amount_expected = int(int(self.number_of_cases) * int(units_per_case))
        self.amount_received = int((int(self.sample_case_count1) + int(self.sample_case_count2) + int(self.sample_case_count3))/3*int(self.number_of_cases))
        self.over_under_received = int(self.amount_received) - int(self.amount_expected)
        self.total_rejections = float((int(self.sample_case_rejections1) + int(self.sample_case_rejections2) + int(self.sample_case_rejections3))/3*int(self.number_of_cases))
        self.rejected_proportion = float(int(self.total_rejections) / int(self.amount_received))

In [48]:
# bring in the data from the shipments file
def read_shipments(file_name):
    global shipments
    shipments = []
    with open(file_name, 'r') as f:
        reader = csv.reader(f)
        next(reader) # skip headers
        for row in reader:
            date_received, ingredient_name, supplier, number_of_cases, units_per_case, \
                 sample_case_count1, sample_case_count2, sample_case_count3, \
                 sample_case_rejections1, sample_case_rejections2, sample_case_rejections3 = row[0],row[1],\
            row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10]

            shipments.append(Shipments(date_received, ingredient_name, supplier, number_of_cases, units_per_case, \
                 sample_case_count1, sample_case_count2, sample_case_count3, \
                 sample_case_rejections1, sample_case_rejections2, sample_case_rejections3))
    return shipments

In [49]:
read_shipments('/content/3967_Module 5_Data File_rejections.csv')

In [50]:
len(shipments)

17

In [51]:
# identify values that are greater than acceptable range

In [52]:
for i in range(17):
  print(shipments[i].amount_received)

1016
9466
2500
996
709
750
960
754
600
1826
450
80
1800
1800
4040
633
333


In [53]:
for i in range(17):
  print(shipments[i].amount_expected)

1000
10000
2500
1000
700
750
960
780
600
1800
450
80
1800
1800
4000
700
350


In [60]:
for i in range(17):
  if ((shipments[i].over_under_received - shipments[i].total_rejections)/shipments[i].amount_expected) < -0.05:
    print(shipments[i].date_received, shipments[i].ingredient_name, shipments[i].supplier)
  #print((shipments[i].over_under_received - shipments[i].total_rejections)/shipments[i].amount_expected)

10-Jun-21 tomato_cherry Tomato Land
10-Jun-21 cucumber_english ABC Vegetables
11-Jun-21 bananas_organic ABC Vegetables
13-Jun-21 grapes_red Cali Fruits
13-Jun-21 grapes_green Cali Fruits


In [82]:
def calculate_total_shortage(i):
  if i.over_under_received:
    total_shortage = 1 - (i.amount_received - i.total_rejections) / i.amount_expected
    return total_shortage * 100
  else:
    return 0

In [76]:
calculate_total_shortage(shipments[0])

2.733333333333332

In [65]:
print(shipments[0].ingredient_name, shipments[0].supplier, shipments[0].total_rejections, shipments[0].over_under_received, shipments[0].amount_expected, shipments[0].amount_received)

tomato_cherry ABC Vegetables 43.33333333333333 16 1000 1016


In [66]:
43/1000

0.043

In [67]:
43/1016

0.04232283464566929

In [74]:
1 - (1016 - 43)/1000

0.027000000000000024

In [88]:
shortages = {}

for i in shipments:
  if calculate_total_shortage(i) > 5:
    key = (i.date_received, i.supplier)

    if key not in shortages:
      shortages[key] = []

    shortages[key].append((i.ingredient_name, calculate_total_shortage(i)))

In [89]:
shortages

{('10-Jun-21', 'Tomato Land'): [('tomato_cherry', 7.673333333333343)],
 ('10-Jun-21', 'ABC Vegetables'): [('cucumber_english', 5.380952380952375)],
 ('11-Jun-21', 'ABC Vegetables'): [('bananas_organic', 6.666666666666665)],
 ('13-Jun-21', 'Cali Fruits'): [('grapes_red', 13.38095238095237),
  ('grapes_green', 17.23809523809523)]}

In [91]:
for (date, supplier), value in shortages.items():
  print(date)
  print(supplier)
  print(value)

10-Jun-21
Tomato Land
[('tomato_cherry', 7.673333333333343)]
10-Jun-21
ABC Vegetables
[('cucumber_english', 5.380952380952375)]
11-Jun-21
ABC Vegetables
[('bananas_organic', 6.666666666666665)]
13-Jun-21
Cali Fruits
[('grapes_red', 13.38095238095237), ('grapes_green', 17.23809523809523)]


In [92]:
for i in shipments:
  print(i.supplier)

ABC Vegetables
Tomato Land
Tomato Land
ABC Vegetables
ABC Vegetables
Dill Farms
Banana Farms Inc
ABC Vegetables
Banana Farms Inc
Orchard Inc
Orchard Inc
Orchard Inc
Apple Farm
Cali Fruits
ABC Vegetables
Cali Fruits
Cali Fruits


In [100]:
output_folder = '/content/output'
for (date, supplier), shorts in shortages.items():
  supplier_clean = supplier.replace(' ', '_')
  date_clean = date.replace('-','_')

  file_name = f"{supplier_clean}_{date_clean}.txt"
  file_path = f"{output_folder}/{file_name}"

  # add content the file

  email_lines = [
      f"Summary of short products for {supplier} regarding shipment on date {date}\n",
      f"{'Product Name':<20}{'Short Percentage':<20}\n",
      f"{'-'*40}\n"] + [f'{product_name:<20}{shorts:<20}\n' for product_name, shorts in shorts]


  with open(file_path, 'w') as f:
    f.writelines(email_lines)

  print(f"written_file: {file_path}")

written_file: /content/output/Tomato_Land_10_Jun_21.txt
written_file: /content/output/ABC_Vegetables_10_Jun_21.txt
written_file: /content/output/ABC_Vegetables_11_Jun_21.txt
written_file: /content/output/Cali_Fruits_13_Jun_21.txt
